In [140]:
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
%matplotlib inline
import re
import pandas as pd

PATH = 'C:\\Users\\Inmetrics\\Documents\\vivo\\Help\\jvms\\JVMs\\Srv02\\'
FILE_NAME_EXCEL = 'Srv01_Parse.xlsx'

In [141]:
data = pd.read_csv(PATH + 'access.log',
                   sep='\s',
                   engine='python', 
                   na_values='-', 
                   header=None,
                   names=['cip', 'date', 'time', 'csmethod', 'csuri' ,'status' ,'bytes', 'timetaken'],
                   )

In [142]:

req_timetaken = data.groupby(by=['csuri'])['timetaken'].mean().reset_index().sort_values(by='csuri')

req_qtd = data.groupby(by=['csuri'])['csmethod'].count().reset_index().sort_values(by='csuri')

dados_request = {'URI':[], 'Media TameTaken': [], 'Qtd' : [], 'Hist' : []}


for val in req_timetaken['csuri'].values:
    dados_request['URI'].append(val)

for val in req_timetaken['timetaken'].values:
    dados_request['Media TameTaken'].append(val)

for ad in req_qtd['csmethod'].values:
    dados_request['Qtd'].append(ad)
    
for index in range(len(dados_request['Media TameTaken'])):
    if dados_request['Media TameTaken'][index] <= 0.5:
        dados_request['Hist'].insert(index, '0s - 0,5s')
    elif dados_request['Media TameTaken'][index] <= 1:
        dados_request['Hist'].insert(index, '0,5s - 1s')
    elif dados_request['Media TameTaken'][index] <= 1.5:
        dados_request['Hist'].insert(index, '1s - 1,5s')
    elif dados_request['Media TameTaken'][index] <= 2:
        dados_request['Hist'].insert(index, '1.5s - 2s')
    else:
        dados_request['Hist'].insert(index, 'Maior 2s')    
         

df = pd.DataFrame(dados_request, columns=['URI','Media TameTaken', 'Qtd', 'Hist' ])
df.sort_values(by='Qtd',ascending=0).head()

,URI,Media TameTaken,Qtd,Hist
1984,/check-ping-web/index.jsp,0.000851,216708,"0s - 0,5s"
4899,/soa-infra/services/SincronismoNotificacaoVivo...,0.353216,29005,"0s - 0,5s"
4881,/soa-infra/services/BillInvoiceManagement/Crea...,0.213196,7410,"0s - 0,5s"
613,/atualizarDadosProcesso/AtualizarDadosProcesso...,0.089595,4680,"0s - 0,5s"
4894,/soa-infra/services/ServiceConfigurationActiva...,0.214476,4228,"0s - 0,5s"


In [143]:
writer = pd.ExcelWriter(PATH + FILE_NAME_EXCEL, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Plan1')
writer.save()


